In [1]:
import sys
sys.path.append('../../../Fedot.Industrial')

import os
import numpy as np
from torch.utils.data import Subset
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
from core.architecture.datasets.splitters import get_dataset_mean_std, split_data, undersampling, dataset_info

DATASETS_ROOT = '/media/n31v/data/datasets'

## MNIST
mean=(0.13066,), std=(0.3081,)

In [13]:
from torchvision.datasets import MNIST

mnist_ds = MNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
)
mean, std = get_dataset_mean_std(mnist_ds)
dataset_info(mnist_ds)
print(f'{mean=}, {std=}')

100%|██████████| 60000/60000 [00:04<00:00, 14236.07it/s]

Class 0 contains 5923 samples.
Class 1 contains 6742 samples.
Class 2 contains 5958 samples.
Class 3 contains 6131 samples.
Class 4 contains 5842 samples.
Class 5 contains 5421 samples.
Class 6 contains 5918 samples.
Class 7 contains 6265 samples.
Class 8 contains 5851 samples.
Class 9 contains 5949 samples.
mean = (0.13066047797803165,), std = (0.3081078048756658,)


In [ ]:
mnist_ds = undersampling(mnist_ds, n=5000)

folds = []
for i in range(5):
    f1, f2 = split_data(mnist_ds, 2)
    folds.append(np.array([mnist_ds.indices[f1], mnist_ds.indices[f2]]))
folds = np.array(folds)
np.save(os.path.join(DATASETS_ROOT, 'MNIST', 'folds'), folds)
print(folds.shape)

In [ ]:
folds = np.load(os.path.join(DATASETS_ROOT, 'MNIST', 'folds.npy'))

mnist_ds = MNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
)

for i in range(5):
    fold1 = Subset(dataset=mnist_ds, indices=folds[i, 0, :])
    dataset_info(fold1)
    fold2 = Subset(dataset=mnist_ds, indices=folds[i, 1, :])
    dataset_info(fold2)

## FashionMNIST
mean=(0.286,), std=(0.353,)

In [ ]:
from torchvision.datasets import FashionMNIST

fmnist_ds = FashionMNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
    download=True,
)
mean, std = get_dataset_mean_std(fmnist_ds)
dataset_info(fmnist_ds)
print(f'{mean=}, {std=}')

In [ ]:
fmnist_ds = undersampling(fmnist_ds, n=5000)

folds = []
for i in range(5):
    f1, f2 = split_data(fmnist_ds, 2)
    folds.append(np.array([fmnist_ds.indices[f1], fmnist_ds.indices[f2]]))
folds = np.array(folds)
np.save(os.path.join(DATASETS_ROOT, 'FashionMNIST', 'folds'), folds)
print(folds.shape)

In [ ]:
folds = np.load(os.path.join(DATASETS_ROOT, 'FashionMNIST', 'folds.npy'))

mnist_ds = FashionMNIST(
    root=DATASETS_ROOT,
    transform=ToTensor(),
)

for i in range(5):
    fold1 = Subset(dataset=mnist_ds, indices=folds[i, 0, :])
    dataset_info(fold1)
    fold2 = Subset(dataset=mnist_ds, indices=folds[i, 1, :])
    dataset_info(fold2)

## CIFAR10
mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616)

In [16]:
from torchvision.datasets import CIFAR10

cifar10_ds = CIFAR10(
    root=os.path.join(DATASETS_ROOT, 'CIFAR10'),
    transform=ToTensor(),
)
mean, std = get_dataset_mean_std(cifar10_ds)
dataset_info(cifar10_ds)
print(f'{mean=}, {std=}')

100%|██████████| 50000/50000 [00:04<00:00, 11767.78it/s]

Class 0 contains 5000 samples.
Class 1 contains 5000 samples.
Class 2 contains 5000 samples.
Class 3 contains 5000 samples.
Class 4 contains 5000 samples.
Class 5 contains 5000 samples.
Class 6 contains 5000 samples.
Class 7 contains 5000 samples.
Class 8 contains 5000 samples.
Class 9 contains 5000 samples.
mean = (0.4913996927399561, 0.4821584222899936, 0.4465309280202538), std = (0.24703223297351337, 0.2434851287896555, 0.26158784042441807)


In [ ]:
folds = []
for i in range(5):
    folds.append(np.array(split_data(cifar10_ds, 2)))
folds = np.array(folds)
np.save(os.path.join(DATASETS_ROOT, 'CIFAR10', 'folds'), folds)
print(folds.shape)

In [ ]:
folds = np.load(os.path.join(DATASETS_ROOT, 'CIFAR10', 'folds.npy'))

cifar10_ds = CIFAR10(
    root=os.path.join(DATASETS_ROOT, 'CIFAR10'),
    transform=ToTensor(),
)

for i in range(5):
    fold1 = Subset(dataset=cifar10_ds, indices=folds[i, 0, :])
    dataset_info(fold1)
    fold2 = Subset(dataset=cifar10_ds, indices=folds[i, 1, :])
    dataset_info(fold2)

## ImageFolder

In [2]:
from typing import Optional
from torchvision.datasets import ImageFolder

def check_dataset(dataset: str):
    folds = np.load(os.path.join(DATASETS_ROOT, dataset, 'folds.npy'))
    ds = ImageFolder(
        root=os.path.join(DATASETS_ROOT, dataset),
        transform=ToTensor(),
    )

    for i in range(5):
        fold1 = Subset(dataset=ds, indices=folds[i, 0, :])
        dataset_info(fold1, verbose=True)
        fold2 = Subset(dataset=ds, indices=folds[i, 1, :])
        dataset_info(fold2, verbose=True)

def prepare_dataset(dataset: str, check: bool = True, n: Optional[int] = None):
    ds = ImageFolder(
        root=os.path.join(DATASETS_ROOT, dataset),
        transform=ToTensor(),
    )
    mean, std = get_dataset_mean_std(ds)
    classes = dataset_info(ds)
    print('------------------------------------------------------------------')
    print('dataset info:')
    print('------------------------------------------------------------------')
    idx_to_class = {v: k for k, v in ds.class_to_idx.items()}
    for k, v in classes.items():
        print(f"Class {k} {idx_to_class[k]} contains {v} samples.")
    print(f'{mean=}, {std=}')
    print('------------------------------------------------------------------')
    print('undersamling...')
    print('------------------------------------------------------------------')
    if n is None:
        n = min(classes.values())
        n = n if n % 2 == 0 else n - 1
    ds = undersampling(ds, n=n)
    mean, std = get_dataset_mean_std(ds)
    print(f'{mean=}, {std=}')
    folds = []
    for i in range(5):
        f1, f2 = split_data(ds, 2)
        folds.append(np.array([ds.indices[f1], ds.indices[f2]]))
    folds = np.array(folds)
    np.save(os.path.join(DATASETS_ROOT, dataset, 'folds'), folds)
    print(folds.shape)
    if check:
        print('------------------------------------------------------------------')
        print('checking dataset...')
        print('------------------------------------------------------------------')
        check_dataset(dataset)

## Land-Use_Scene_Classification
mean=(0.462, 0.471, 0.440), std=(0.287, 0.279, 0.269)

In [ ]:
prepare_dataset(os.path.join(DATASETS_ROOT, 'Land-Use_Scene_Classification/images'), n=200)

## Minerals 200x200
mean=(0.54, 0.61, 0.51), std=(0.22, 0.23, 0.23)

In [3]:
prepare_dataset(os.path.join(DATASETS_ROOT, 'minerals_21_200'), n=200)

100%|██████████| 30231/30231 [00:40<00:00, 749.75it/s] 


------------------------------------------------------------------
dataset info:
------------------------------------------------------------------
Class 0 01_Almandin contains 1257 samples.
Class 1 02_Amazonit contains 1349 samples.
Class 2 03_Apatit contains 1665 samples.
Class 3 04_Barit contains 1434 samples.
Class 4 05_Berill contains 1459 samples.
Class 5 06_Biotit contains 1371 samples.
Class 6 07_Granat contains 1650 samples.
Class 7 08_Diopsit contains 1499 samples.
Class 8 09_Gold contains 1369 samples.
Class 9 11_Quarz_1 contains 1426 samples.
Class 10 13_Kianit contains 1161 samples.
Class 11 14_Kordierit contains 1619 samples.
Class 12 16_Magnetit contains 1036 samples.
Class 13 17_Microclin contains 1562 samples.
Class 14 18_Muscovit contains 1676 samples.
Class 15 19_Olivin contains 1530 samples.
Class 16 20_Pirit contains 1148 samples.
Class 17 21_Fluorit contains 1344 samples.
Class 18 22_Chromshpinelid contains 1589 samples.
Class 19 23_Cavorit contains 1286 samples.


100%|██████████| 30231/30231 [00:21<00:00, 1433.45it/s]


New size of any class 200 samples.


100%|██████████| 4200/4200 [00:03<00:00, 1208.01it/s]


...

Class 0 contains 100 samples.
Class 1 contains 100 samples.
Class 2 contains 100 samples.
Class 3 contains 100 samples.
Class 4 contains 100 samples.
Class 5 contains 100 samples.
Class 6 contains 100 samples.
Class 7 contains 100 samples.
Class 8 contains 100 samples.
Class 9 contains 100 samples.
Class 10 contains 100 samples.
Class 11 contains 100 samples.
Class 12 contains 100 samples.
Class 13 contains 100 samples.
Class 14 contains 100 samples.
Class 15 contains 100 samples.
Class 16 contains 100 samples.
Class 17 contains 100 samples.
Class 18 contains 100 samples.
Class 19 contains 100 samples.
Class 20 contains 100 samples.


100%|██████████| 2100/2100 [00:01<00:00, 1453.28it/s]


Class 0 contains 100 samples.
Class 1 contains 100 samples.
Class 2 contains 100 samples.
Class 3 contains 100 samples.
Class 4 contains 100 samples.
Class 5 contains 100 samples.
Class 6 contains 100 samples.
Class 7 contains 100 samples.
Class 8 contains 100 samples.
Class 9 contains 100 samples.
Class 10 contains 100 samples.
Class 11 contains 100 samples.
Class 12 contains 100 samples.
Class 13 contains 100 samples.
Class 14 contains 100 samples.
Class 15 contains 100 samples.
Class 16 contains 100 samples.
Class 17 contains 100 samples.
Class 18 contains 100 samples.
Class 19 contains 100 samples.
Class 20 contains 100 samples.


100%|██████████| 2100/2100 [00:01<00:00, 1448.43it/s]

Class 0 contains 100 samples.
Class 1 contains 100 samples.
Class 2 contains 100 samples.
Class 3 contains 100 samples.
Class 4 contains 100 samples.
Class 5 contains 100 samples.
Class 6 contains 100 samples.
Class 7 contains 100 samples.
Class 8 contains 100 samples.
Class 9 contains 100 samples.
Class 10 contains 100 samples.
Class 11 contains 100 samples.
Class 12 contains 100 samples.
Class 13 contains 100 samples.
Class 14 contains 100 samples.
Class 15 contains 100 samples.
Class 16 contains 100 samples.
Class 17 contains 100 samples.
Class 18 contains 100 samples.
Class 19 contains 100 samples.
Class 20 contains 100 samples.
